In [ ]:
# Object serialization example

import pickle

a = {'hello': 'world'}

with open('filename.pickle', 'wb') as handle:
    pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('filename.pickle', 'rb') as handle:
    b = pickle.load(handle)

print(a == b)

In [3]:
cd ..


/Users/jacek/projects/cqa-sem-eval


In [4]:
# prepare data
import gensim
import re
from bs4 import BeautifulSoup
# import word2vec_model.word2vec_utils as word2vec

def load(file_name):
    with open(file_name, 'r', encoding="utf8") as myfile:
        return BeautifulSoup(myfile.read(), "xml")
    
def remove_subject_from_question(question):
    a =  re.sub('.+\/\/\ ', '', question);
    if (a != question):
        print(question + " " + a + "\n")
    return a

def subtask_B_word2vec_dataset(xml_file):
    print("Loading word2vec dataset...")
    
    dataset = subtask_A_raw_dataset(xml_file)
    

def label_to_class(label):
    if label == "PerfectMatch":
        return 0
    if label == "Relevant":
        return 1
    if label == "Irrelevant":
        return 2
    
def label_to_class_2(label):
    if label == "Irrelevant":
        return 0
    else:
        return 1
    
def load_file_subject_body_merged(xml_file):
    print("Loading raw subject - body dataset")
    
    soup = load(xml_file)
    original_questions = soup.findAll("OrgQuestion")
    data_set = []
    
    for original_question in original_questions:
        related_questions = original_question.findAll("RelQuestion")
        
        for related_question in related_questions:
            
            original_question_text = original_question.OrgQSubject.text + ". " + original_question.OrgQBody.text
            related_question_text = related_question.RelQSubject.text + ". " + related_question.RelQBody.text
            
            data_set_sample = dict([("original_question", remove_subject_from_question(original_question_text)),
                                    ("related_question", related_question_text),
                                    ("relevance", label_to_class_2(related_question['RELQ_RELEVANCE2ORGQ']))])
            
#             print(data_set_sample)
            data_set.append(data_set_sample)
            
    print("Loading raw data set [DONE]")
    return data_set




In [5]:
test_file_input = "data/SemEval2017-task3-English-test-input.xml"
# file = "data/SemEval2016-Task3-CQA-QL-train-part2.xml"
# asd = load_file_subject_body_merged(test_file_input)
# train2_body_pairs = load_file_body(file)
# train2_body_pairs = load_file_subject(file)

asd = load_test_file_subject_body_merged(test_file_input)


Loading raw subject - body dataset
Loading raw data set [DONE]


In [ ]:
train2_body_pairs

In [7]:
import json
with open('subtask_B/data/test_original_related_subject_body_merged.json', 'w') as outfile:
    json.dump(asd, outfile)

In [ ]:
# generowanie z jsona obiektow X i Y 

with open('original_related_body_relevance_pairs.json', 'r') as file:
    data_set = json.load(file)
    

In [ ]:
GOOGLE_TRAINED_VECTORS = 'cqa-sem-eval/data/GoogleNews-vectors-negative300.bin'

def data_to_word2vec_3_classes():
    word2vec_model = load_word2vec_model(GOOGLE_TRAINED_VECTORS)
    result = []
    for sample in data_set:
        org_question_vector = word2vec.sentence_vectors_mean(
            word2vec.sentence2vectors(sample["original_question"], word2vec_model, exclude_stopwords=True, to_lower_case=True))
        rel_question_vector = word2vec.sentence_vectors_mean(
            word2vec.sentence2vectors(sample["related_question"], word2vec_model, to_lower_case=True, exclude_stopwords=True))
        if len(org_question_vector) == 0 or len(rel_question_vector) == 0:
            continue
        transformed_sample = dict([
            ("org_question", org_question_vector),
            ("rel_question", rel_question_vector),
            ("relevance", label_to_class(sample["relevance"]))])
        result.append(transformed_sample)
    print("Loading word2vec data set [DONE]")
    return result


def data_to_word2vec_2_classes():
    word2vec_model = load_word2vec_model(GOOGLE_TRAINED_VECTORS)
    result = []
    for sample in data_set:
        org_question_vector = word2vec.sentence_vectors_mean(
            word2vec.sentence2vectors(sample["original_question"], word2vec_model, exclude_stopwords=True, to_lower_case=True))
        rel_question_vector = word2vec.sentence_vectors_mean(
            word2vec.sentence2vectors(sample["related_question"], word2vec_model, to_lower_case=True, exclude_stopwords=True))
        if len(org_question_vector) == 0 or len(rel_question_vector) == 0:
            continue
        transformed_sample = dict([
            ("org_question", org_question_vector),
            ("rel_question", rel_question_vector),
            ("relevance", label_to_class_2(sample["relevance"]))])
        result.append(transformed_sample)
    print("Loading word2vec data set [DONE]")
    return result

In [ ]:
word2_vec_first_dataset = data_to_word2vec_3_classes()

In [ ]:
import nltk
import numpy as np
import gensim
import sklearn
import math
import os
from stop_words import get_stop_words


def load_word2vec_model(model_name):
    print("Loading word2vec model {}".format(model_name))
    path = os.path.dirname(os.path.abspath(os.getcwd()))
    model = None
    if model_name == "Q1_model" or model_name == "SemEval2016-Task3-CQA-QL-dev_model":
        model = gensim.models.Word2Vec.load(path + "/" + model_name)
    elif model_name == "cqa-sem-eval/data/GoogleNews-vectors-negative300.bin":
        model = gensim.models.KeyedVectors.load_word2vec_format(path + "/" + model_name,
                                                                binary=True)
    else:
        raise Exception("Unknown word2vec model {}".format(model_name))
    print("Loading word2vec model {} [DONE]".format(model_name))
    return model

In [ ]:
load_word2vec_model(GOOGLE_TRAINED_VECTORS)

In [ ]:
input_data = data_to_word2vec_2_classes()

In [ ]:
import keras
X = []
Y = []

for sample in input_data:
    org_question_vec = sample["org_question"]
    rel_question_vec = sample["rel_question"]
    INPUT_DIM = len(org_question_vec) + len(rel_question_vec)
    concatenated_vec = np.concatenate((org_question_vec, rel_question_vec), axis=0)
    X.append(concatenated_vec)
    Y.append(sample["relevance"])
X = np.reshape(X, (-1, INPUT_DIM))
Y = np.reshape(Y, (-1, 1))
Y = keras.utils.to_categorical(Y, num_classes=2)

In [ ]:
import pickle

# obj0, obj1, obj2 are created here...

# Saving the objects:
with open('org_rel_body_2_categories.pkl', 'wb') as f:
    pickle.dump([X, Y], f)

# # Getting back the objects:
# with open('objs.pkl') as f:  # Python 3: open(..., 'rb')
#     obj0, obj1, obj2 = pickle.load(f)

In [ ]:
with open('org_rel_body_2_categories.pkl', 'rb') as f:
    X, Y = pickle.load(f)

'/Users/jacek/projects/Untitled Folder'